In [1]:
from pathlib import Path
import joblib

import pandas as pd

In [2]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"
assert DATA_DIR.exists()

DATA_PATH = DATA_DIR / "base_indices_2005-2023.csv"

In [5]:
df = joblib.load(DATA_DIR / "base_indices.pkl")

In [6]:
df = df.drop(
    [
        "Nombre de la Sede",
        "Orden Geográfico de la Región (Norte aSur)",
        "Mención o Especialidad",
        "idgenerocarrera",
        "Cód. Campus",
        "Cód. Sede",
        "Códgo SIES",
    ],
    axis=1,
)

In [ ]:
df